# Reddit Scraper für vergangene Daten 

## Import 

In [68]:
import praw
import pandas as pd
from datetime import datetime, timedelta
import os
import psaw as ps
from dotenv import load_dotenv
import time
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
import torch


In [69]:
# Lade die .env-Datei
dotenv_loaded = load_dotenv("zugang_reddit.env")  # Falls die Datei anders heißt, anpassen
# Prüfe, ob die Datei geladen wurde
print(f".env geladen? {dotenv_loaded}")


.env geladen? True


In [70]:
reddit = praw.Reddit(
    client_id=os.getenv("CLIENT_ID"),
    client_secret=os.getenv("CLIENT_SECRET"),
    user_agent=os.getenv("USER_AGENT")
)

print("Reddit API erfolgreich verbunden!")


Reddit API erfolgreich verbunden!


In [71]:
try:
    subreddit = reddit.subreddit("CryptoCurrency")
    for post in subreddit.hot(limit=5):
        print(f"Title: {post.title}, Score: {post.score}")
except Exception as e:
    print(f"Fehler beim Abrufen der Subreddit-Daten: {e}")


Title: [AMA] Giveaway with Portal to Bitcoin: Making bridges, wrapped coins, and external custody obsolete - Feb 12, Score: 30
Title: Daily Crypto Discussion - February 13, 2025 (GMT+0), Score: 11
Title: Poor vs Rich vs Crypto Bro, Score: 866
Title: 6 men kidnapped Chicago family, forcing $15M crypto transfer, Score: 168
Title: No, no. He's got a point, Score: 3488


Cryptos und Subreddits 

In [72]:
crypto_terms = {
    # 🔹 Top Coins
    "Bitcoin": ["bitcoin", "btc", "sats", "bitcoin network"],
    "Ethereum": ["ethereum", "eth", "ether", "ethereum 2.0", "eth 2.0"],
    "Wrapped Ethereum": ["wrapped ethereum", "weth"],
    "Solana": ["solana", "sol", "sol coin"],
    "Avalanche": ["avalanche", "avax"],
    "Polkadot": ["polkadot", "dot"],
    "Near Protocol": ["near protocol", "near"],
    "Polygon": ["polygon", "matic"],
    "XRP": ["xrp", "ripple"],
    "Cardano": ["cardano", "ada"],
    "Cronos": ["cronos", "cro"],
    "Vulcan Forged PYR": ["vulcan forged", "pyr"],
    "Chiliz": ["chiliz", "chz"],
    "Illuvium": ["illuvium", "ilv"],
    "Ronin": ["ronin", "ron"],
    "Band Protocol": ["band protocol", "band"],
    "Optimism": ["optimism", "op"],
    "Celestia": ["celestia", "tia"],
    "Numerai": ["numerai", "nmr"],
    "Aethir": ["aethir", "ath"],
    "Sui": ["sui"],
    "Hyperliquid": ["hyperliquid", "hyp"],
    "Robinhood Coin": ["robinhood", "hood"],
    "Trump Coin": ["trump coin"],
    "USD Coin": ["usd coin", "usdc"],
    "Binance Coin": ["binance", "bnb"],
    "Litecoin": ["litecoin", "ltc"],
    "Dogecoin": ["dogecoin", "doge"],
    "Tron": ["tron", "trx"],
    "Aave": ["aave"],
    "Hedera": ["hedera", "hbar"],
    "Filecoin": ["filecoin", "fil"],
    "Cosmos": ["cosmos", "atom"],
    "Gala": ["gala"],
    "The Sandbox": ["sandbox", "sand"],
    "Audius": ["audius", "audio"],
    "Render": ["render", "rndr"],
    "Kusama": ["kusama", "ksm"],
    "VeChain": ["vechain", "vet"],
    "Chainlink": ["chainlink", "link"],
    "Berachain": ["berachain", "bera"],
    "TestCoin": ["testcoin", "test"],

    # 🔹 Meme-Coins
    "Shiba Inu": ["shiba inu", "shib"],
    "Pepe": ["pepe"],
    "Floki Inu": ["floki inu", "floki"],
    "Bonk": ["bonk"],
    "Wojak": ["wojak"],
    "Mog Coin": ["mog"],
    "Doge Killer (Leash)": ["leash"],
    "Baby Doge Coin": ["baby doge", "babydoge"],
    "Degen": ["degen"],
    "Toshi": ["toshi"],
    "Fartcoin": ["fartcoin"],
    "Banana": ["banana"],
    "Kabosu": ["kabosu"],
    "Husky": ["husky"],
    "Samoyedcoin": ["samoyedcoin", "samo"],
    "Milkbag": ["milkbag"],

    # 🔹 New Coins
    "Arbitrum": ["arbitrum", "arb"],
    "Starknet": ["starknet", "strk"],
    "Injective Protocol": ["injective", "inj"],
    "Sei Network": ["sei"],
    "Aptos": ["aptos", "apt"],
    "EigenLayer": ["eigenlayer", "eigen"],
    "Mantle": ["mantle", "mnt"],
    "Immutable X": ["immutable x", "imx"],
    "Ondo Finance": ["ondo"],
    "Worldcoin": ["worldcoin", "wld"],
    "Aerodrome": ["aerodrome", "aero"],
    "Jupiter": ["jupiter", "jup"],
    "THORChain": ["thorchain", "rune"],
    "Pendle": ["pendle"],
    "Kujira": ["kujira", "kuji"],
    "Noble": ["noble"],
    "Stride": ["stride", "strd"],
    "Dymension": ["dymension", "dym"],
    "Seamless Protocol": ["seamless", "seam"],
    "Blast": ["blast"],
    "Merlin": ["merlin"],
    "Tapioca": ["tapioca"],
    "Arcadia Finance": ["arcadia"],
    "Notcoin": ["notcoin", "not"],
    "Omni Network": ["omni"],
    "LayerZero": ["layerzero", "lz"],
    "ZetaChain": ["zetachain", "zeta"],
    "Friend.tech": ["friendtech"]
}


In [73]:
subreddits = [
    "CryptoCurrency",  # Allgemeine Diskussionen über Kryptowährungen
    "CryptoMarkets",   # Diskussionen über den Kryptomarkt und Preisbewegungen
    "CryptoTrading",   # Fokus auf Trading-Strategien und Analysen
    "Altcoin",         # Diskussionen über Altcoins (alle Kryptowährungen außer Bitcoin)
    "DeFi",            # Decentralized Finance (DeFi) und Projekte
    "BitcoinBeginners",# Für Anfänger in der Krypto-Welt
    "cryptotechnology", # Fokus auf die zugrunde liegende Blockchain-Technologie
    "cryptocurrencies", # Allgemeine Diskussionen über Kryptowährungen
    "Satoshistreetsbets", # Krypto-Wetten und Spekulationen
    "Binance",        # Diskussionen über die Binance-Plattform  
    "Bitcoin",
    "ethtrader"
]

## Scraping 

Scraping Funktionen

In [74]:
# 🔹 Funktion für neue Posts mit Pagination
def get_all_new_posts(subreddit, start_timestamp, end_timestamp):
    all_posts = []
    after = None

    while True:
        print(f"🔄 Lade neue Posts aus r/{subreddit}... (Aktuell: {len(all_posts)})")

        try:
            new_posts = list(reddit.subreddit(subreddit).new(limit=100, params={"after": after}))
        except praw.exceptions.APIException as e:
            print(f"⚠️ API-Fehler: {e}")
            time.sleep(60)
            continue

        if not new_posts:
            break

        for post in new_posts:
            if start_timestamp <= post.created_utc <= end_timestamp:
                all_posts.append(post)

        after = new_posts[-1].fullname
        time.sleep(2)  # Wartezeit für Rate-Limit

    print(f"✅ Abgeschlossen: {len(all_posts)} neue Posts aus r/{subreddit} geladen!")
    return all_posts

# 🔹 Haupt-Scraper
def scrape_reddit(start_date, end_date):
    start_timestamp = int(start_date.timestamp())
    end_timestamp = int(end_date.timestamp())

    posts_data = []
    comments_data = []

    for subreddit_name in subreddits:
        print(f"🔎 Suche in r/{subreddit_name}...")

        # 🔹 ALLE neuen Posts holen
        new_posts = get_all_new_posts(subreddit_name, start_timestamp, end_timestamp)

        for post in new_posts:
            detected_cryptos = [
                crypto for crypto, terms in crypto_terms.items() 
                if any(term in post.title.lower() for term in terms)
            ]

            # 📝 **Speicherung der Posts**
            posts_data.append({
                "post_id": post.id,
                "subreddit": subreddit_name,
                "title": post.title.lower(),
                "selftext": (post.selftext or "").lower(),
                "author": str(post.author),
                "created_utc": datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                "score": post.score,
                "num_comments": post.num_comments,
                "detected_crypto": detected_cryptos
            })

            # 🔹 Kommentare abrufen (ALLE speichern, nicht nur mit Krypto)
            try:
                time.sleep(1)  # Wartezeit für API-Limit
                post.comments.replace_more(limit=5)  # Begrenzung auf relevante Kommentare

                for comment in post.comments.list():
                    comment_body = (comment.body or "").lower()

                    detected_cryptos_comment = [
                        crypto for crypto, terms in crypto_terms.items() 
                        if any(term in comment_body for term in terms)
                    ]

                    # 💬 **Speicherung der Kommentare**
                    comments_data.append({
                        "comment_id": comment.id,
                        "post_id": post.id,
                        "subreddit": subreddit_name,
                        "author": str(comment.author),
                        "created_utc": datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                        "score": comment.score,
                        "selftext": comment_body,
                        "detected_crypto": detected_cryptos_comment
                    })

            except praw.exceptions.APIException as e:
                print(f"⚠️ API-Fehler beim Abrufen von Kommentaren: {e}")
                time.sleep(30)  # Exponentielle Backoff-Strategie

    # 🔹 DataFrames erstellen
    df_posts = pd.DataFrame(posts_data)
    df_comments = pd.DataFrame(comments_data)

    print(f"✅ Scrape abgeschlossen: {len(df_posts)} Posts und {len(df_comments)} Kommentare gespeichert.")

    return df_posts, df_comments

In [75]:
# 🔹 Prozess für den einmaligen Scrape
start_of_period = datetime(2024, 11, 1)
now = datetime.now()

print("🚀 Starte Scraping-Prozess...")
df_posts, df_comments = scrape_reddit(start_of_period, now)

🚀 Starte Scraping-Prozess...
🔎 Suche in r/CryptoCurrency...
🔄 Lade neue Posts aus r/CryptoCurrency... (Aktuell: 0)
🔄 Lade neue Posts aus r/CryptoCurrency... (Aktuell: 100)
🔄 Lade neue Posts aus r/CryptoCurrency... (Aktuell: 200)
🔄 Lade neue Posts aus r/CryptoCurrency... (Aktuell: 300)
🔄 Lade neue Posts aus r/CryptoCurrency... (Aktuell: 400)
🔄 Lade neue Posts aus r/CryptoCurrency... (Aktuell: 500)
🔄 Lade neue Posts aus r/CryptoCurrency... (Aktuell: 600)
🔄 Lade neue Posts aus r/CryptoCurrency... (Aktuell: 700)
🔄 Lade neue Posts aus r/CryptoCurrency... (Aktuell: 800)
🔄 Lade neue Posts aus r/CryptoCurrency... (Aktuell: 805)
✅ Abgeschlossen: 805 neue Posts aus r/CryptoCurrency geladen!
🔎 Suche in r/CryptoMarkets...
🔄 Lade neue Posts aus r/CryptoMarkets... (Aktuell: 0)
🔄 Lade neue Posts aus r/CryptoMarkets... (Aktuell: 99)
🔄 Lade neue Posts aus r/CryptoMarkets... (Aktuell: 199)
🔄 Lade neue Posts aus r/CryptoMarkets... (Aktuell: 299)
🔄 Lade neue Posts aus r/CryptoMarkets... (Aktuell: 399)
🔄 L

## Clean

In [76]:
def clean_data(df_posts, df_comments, comment_threshold=500):# Anpassbarer Schwellenwert für Kommentare pro Nutzer
    # 1. Duplikate entfernen
    df_posts = df_posts.drop_duplicates(subset=["post_id"])
    df_comments = df_comments.drop_duplicates(subset=["comment_id"])
    
    # 2. Fehlende Werte behandeln
    df_posts['selftext'] = df_posts['selftext'].fillna('')  # Fehlende Posttexte auffüllen
    df_comments['body'] = df_comments['body'].fillna('')  # Fehlende Kommentare auffüllen
    
    # 3. Zeitstempel konvertieren
    df_posts['created_utc'] = pd.to_datetime(df_posts['created_utc'])
    df_comments['created_utc'] = pd.to_datetime(df_comments['created_utc'])

    # 4. Datum & Uhrzeit in separate Spalten aufteilen (Daten normalisieren)
    df_posts["date"] = df_posts["created_utc"].dt.date  # YYYY-MM-DD
    df_posts["time"] = df_posts["created_utc"].dt.time  # HH:MM:SS

    df_comments["date"] = df_comments["created_utc"].dt.date
    df_comments["time"] = df_comments["created_utc"].dt.time

    # 5. Original `created_utc`-Spalte entfernen
    df_posts.drop(columns=["created_utc"], inplace=True)
    df_comments.drop(columns=["created_utc"], inplace=True)

    # 6. Filterung nach Qualität (Spam oder irrelevante Daten entfernen)
    df_posts = df_posts[df_posts['score'] > 0]  # Posts mit negativem Score entfernen
    df_comments = df_comments[df_comments['score'] > 0]  # Kommentare mit negativem Score entfernen

    # 7. Entferne bekannte Bot-Accounts
    bot_accounts = ["AutoModerator", "coinfeeds-bot", "devCheckingIn"]
    df_comments = df_comments[~df_comments["author"].isin(bot_accounts)]

    # 8. Entferne Nutzer mit übermäßigen Kommentaren
    comment_counts = df_comments["author"].value_counts()
    frequent_users = comment_counts[comment_counts > comment_threshold].index  # Nutzer über Grenze
    df_comments = df_comments[~df_comments["author"].isin(frequent_users)]

    print(f"✅ Daten bereinigt: {df_comments.shape[0]} Kommentare übrig (nach Spam-Filter).")

    return df_posts, df_comments

In [78]:
# Bereinigen der Daten
df_posts_clean, df_comments_clean = clean_data(df_posts, df_comments)

# Überprüfen, wie viele Einträge übrig sind
print(f"Bereinigte Posts: {len(df_posts_clean)}")
print(f"Bereinigte Kommentare: {len(df_comments_clean)}")

KeyError: 'body'

## Model fuer das Sentiment 

In [ ]:
# 🔹 CryptoBERT-Modell laden
MODEL_NAME = "ElKulako/cryptobert"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

# 🔹 Sicherstellen, dass die Spalten "title" und "body" existieren
if "title" not in df_posts_clean.columns or "body" not in df_comments_clean.columns:
    raise ValueError("Fehler: Die CSV-Dateien enthalten nicht die erforderlichen Spalten ('title' für Posts, 'body' für Kommentare)!")

# 🔹 Funktion zur Sentiment-Analyse mit CryptoBERT
def analyze_sentiment(text):
    if not isinstance(text, str) or text.strip() == "":
        return "neutral", 0.0  # Leere Einträge sind neutral

    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)

    scores = F.softmax(outputs.logits, dim=1)[0]
    labels = ["bearish", "neutral", "bullish"]  # CryptoBERT nutzt diese Labels
    sentiment = labels[torch.argmax(scores).item()]
    confidence = scores.max().item()

    return sentiment, confidence

# 🔹 Sentiment für **Posts** berechnen (auf Basis des Titels und ggf. Selbsttextes)
df_posts_clean["sentiment"], df_posts_clean["sentiment_confidence"] = zip(
    *df_posts_clean["title"].map(analyze_sentiment)
)

# Falls es einen Selbsttext gibt, kombinieren wir diesen mit dem Titel für eine genauere Analyse
df_posts_clean["full_text"] = df_posts_clean["title"] + " " + df_posts_clean["selftext"].fillna("")
df_posts_clean["sentiment"], df_posts_clean["sentiment_confidence"] = zip(
    *df_posts_clean["full_text"].map(analyze_sentiment)
)

# 🔹 Sentiment für **Kommentare** berechnen
df_comments_clean["sentiment"], df_comments_clean["sentiment_confidence"] = zip(
    *df_comments_clean["body"].map(analyze_sentiment)
)

# 🔹 Ergebnisse anzeigen
print(f"✅ Sentiment-Analyse abgeschlossen: {len(df_posts_clean)} Posts & {len(df_comments_clean)} Kommentare bewertet.")


AttributeError: 'tuple' object has no attribute 'columns'

## Merge

In [ ]:
# 🔹 Spalte "type" hinzufügen (damit klar ist, ob es ein Post oder Kommentar ist)
df_posts_clean["type"] = "post"
df_comments_clean["type"] = "comment"

# 🔹 Mergen von Posts & Kommentaren 
df_merged = pd.concat([df_posts_clean, df_comments_clean], ignore_index=True)

## Export 

In [ ]:
# Setze den Pfad zu deinem Google Drive Ordner
DRIVE_PATH = "G:/Meine Ablage/reddit/"
POSTS_CSV = os.path.join(DRIVE_PATH, "reddit_posts.csv")
COMMENTS_CSV = os.path.join(DRIVE_PATH, "reddit_comments.csv")
MERGED_CSV = os.path.join(DRIVE_PATH, "reddit_merged.csv")

Funktion zum Export 

In [ ]:
def append_to_csv(df_new, filename, key_column):
    """Hängt neue Daten an eine bestehende CSV an & entfernt Duplikate."""
    file_path = os.path.join(DRIVE_PATH, filename)

    try:
        # Falls Datei existiert, alte Daten einlesen
        if os.path.exists(file_path):
            df_existing = pd.read_csv(file_path, sep="|", encoding="utf-8-sig", on_bad_lines="skip")
            
            # 🔹 Daten zusammenführen & Duplikate nach `key_column` entfernen (neuere Werte behalten)
            df_combined = pd.concat([df_existing, df_new], ignore_index=True).drop_duplicates(subset=[key_column], keep="last")
        else:
            df_combined = df_new  # Falls keine Datei existiert, neue Daten direkt nutzen

        # 🔹 CSV speichern
        df_combined.to_csv(
            file_path,
            index=False,
            sep="|",
            encoding="utf-8-sig",
            lineterminator="\n"
        )
        print(f"✅ Datei erfolgreich aktualisiert: {file_path}")

    except Exception as e:
        print(f"Fehler beim Speichern der Datei {filename}: {e}")

def export_to_drive(df_posts, df_comments, df_merged):
    """Speichert Posts, Kommentare & die gemergte Datei mit Duplikat-Prüfung."""
    try:
        append_to_csv(df_posts, "reddit_posts.csv", key_column="post_id")
        append_to_csv(df_comments, "reddit_comments.csv", key_column="comment_id")
        append_to_csv(df_merged, "reddit_merged.csv", key_column="comment_id")  # Falls Kommentare entscheidend sind

    except Exception as e:
        print(f"Fehler beim Export: {e}")

In [ ]:
# 🔹 Export-Funktion aufrufen
export_to_drive(df_posts_clean, df_comments_clean, df_merged)

✅ Datei erfolgreich gespeichert: G:/Meine Ablage/reddit/reddit_posts.csv
✅ Datei erfolgreich gespeichert: G:/Meine Ablage/reddit/reddit_comments.csv
✅ Datei erfolgreich gespeichert: G:/Meine Ablage/reddit/reddit_merged.csv
